In [103]:
import numpy as np
import json
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


with open("processed_data/sports.json","r") as f:
    sports = json.load(f)

with open("processed_data/finalData4.json","r") as f:
    Data = json.load(f)
    print(len(Data))

with open("processed_data/N0.json","r") as f:
    N0 = json.load(f)

with open("processed_data/finalData2024.json") as f:
    predData = json.load(f)


total_pred_data = []
for key,value in predData.items():
    # print(len(value))
    # print(value)
    if value == None:
        continue
    # if key == "Athletics":
    #     print(value)
        # break
    total_pred_data.extend([b+[key] for b in value])


# print(total_pred_data)
# for total_pred in total_pred_data:
#     print(len(total_pred))
#     if len(total_pred) !=11:
#         print(total_pred)
#         break

# print(total_pred_data)
total_pred_data = np.array(total_pred_data)
print(total_pred_data.shape)
pred_data = total_pred_data[:,:7]

pred_data = pred_data.astype(np.float32)
# print(predData)
print(pred_data.shape)

pred_data = torch.tensor(pred_data, dtype=torch.float32)

    

60
(4063, 12)
(4063, 7)


In [104]:

# 定义MLP模型
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(7, 5)  # 输入层到隐藏层1 (9个输入)

    def forward(self, x):
        # x = F.softmax(self.fc1(x),dim=1)  # 使用ReLU激活函数
        x = F.sigmoid(self.fc1(x))
        return x


In [105]:

torch.manual_seed(42)

dataValue = Data.values()

total_data = list(itertools.chain(*dataValue))
total_data = np.array(total_data)
print(total_data.shape)

total_x = total_data[:,:7].astype(np.float32)
total_y = total_data[:,7].astype(np.float32)

label_encoder = LabelEncoder()
label_encoder.fit([-1, 0, 1, 2, 3])
total_y_encoded = label_encoder.transform(total_y)

X_train, X_test, y_train, y_test = train_test_split(total_x, total_y_encoded, test_size=0.2, random_state=42)


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


model = MLPModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)



# 训练模型
epochs = 100  # 训练50个epoch
for epoch in range(epochs):
    model.train()  # 设置模型为训练模式

    # 清空梯度
    optimizer.zero_grad()

    # 前向传播
    outputs = model(X_train)

    # 计算损失
    loss = criterion(outputs, y_train)

    # 反向传播和优化
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 进行预测（示例）
model.eval()  # 设置模型为评估模式
with torch.no_grad():
    # 假设我们有一个测试样本
    y_pre = model(X_test)
    print(y_pre)
    # 获取预测的类别
    _, predicted = torch.max(y_pre, 1)
    print(type(predicted))
    print(predicted)
    # predicted = predicted.numpy()
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)    
    print(f"Accuracy: {accuracy:.4f}")


    pred2028 = model(pred_data)
    pred_y = model(torch.tensor(total_x, dtype=torch.float32))

pred2028 = pred2028.detach().tolist()
PredData2028 = np.concatenate((total_pred_data,pred2028),axis=1)
np.save("processed_data/PredData2028.npy",PredData2028)

pred_y = pred_y.detach().tolist()
TotalData = np.concatenate((total_data,pred_y),axis=1)
np.save("processed_data/TotalData.npy",TotalData)


(79311, 11)
Epoch [10/100], Loss: 1.6093
Epoch [20/100], Loss: 1.5736
Epoch [30/100], Loss: 1.5403
Epoch [40/100], Loss: 1.5097
Epoch [50/100], Loss: 1.4822
Epoch [60/100], Loss: 1.4577
Epoch [70/100], Loss: 1.4361
Epoch [80/100], Loss: 1.4171
Epoch [90/100], Loss: 1.4004
Epoch [100/100], Loss: 1.3857
tensor([[0.6199, 0.8050, 0.2290, 0.2126, 0.2336],
        [0.6199, 0.8050, 0.2290, 0.2126, 0.2336],
        [0.9066, 0.5902, 0.5488, 0.3832, 0.3917],
        ...,
        [0.7873, 0.7836, 0.3023, 0.2323, 0.2539],
        [0.4538, 0.6811, 0.2817, 0.3063, 0.3165],
        [0.6947, 0.9613, 0.0981, 0.0591, 0.1807]])
<class 'torch.Tensor'>
tensor([1, 1, 0,  ..., 0, 1, 1])
Accuracy: 0.7158
